In [360]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import re
import os
import pickle
import warnings
import wget
import sklearn
import random
import nltk
import spacy
import textblob
import gensim 
import Sastrawi
from gensim.models import Word2Vec
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding, GRU, SimpleRNN
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils.data_utils import pad_sequences
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score, recall_score, precision_score
from nltk.corpus import stopwords
from nltk.tree import Tree
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import PorterStemmer
from collections import Counter, OrderedDict
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from spacy.util import minibatch, compounding
from spacy import load, displacy
from spacy.training.example import Example
from nlp_id import stopword
from unidecode import unidecode
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Hammam\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Hammam\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [361]:
df = pd.read_csv('./tweet.csv')

In [362]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1815 entries, 0 to 1814
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  1815 non-null   int64 
 1   sentimen    1815 non-null   object
 2   tweet       1815 non-null   object
dtypes: int64(1), object(2)
memory usage: 42.7+ KB


In [363]:
df.head()

,Unnamed: 0,sentimen,tweet
0,0,negatif,Kata @prabowo Indonesia tidak dihargai bangsa ...
1,1,netral,"Batuan Langka, Tasbih Jokowi Hadiah dari Habib..."
2,2,netral,"Di era Jokowi, ekonomi Indonesia semakin baik...."
3,3,positif,"Bagi Sumatera Selatan, Asian Games berdampak p..."
4,4,negatif,Negara kita ngutang buat bngun infrastruktur y...


In [364]:
df.tail()

,Unnamed: 0,sentimen,tweet
1810,1810,netral,Negarawan sejati sll bangga dan mengedepankan ...
1811,1811,netral,1. HRS ceramah di Damai Indonesiaku 2. Perekon...
1812,1812,netral,Mari bangun bangsa dgn mendukung perekonomian ...
1813,1813,netral,Bantu majukan perekonomian bangsa bersama Pak ...
1814,1814,netral,Pak @jokowi mengubah cara pandang ekonomi. Kin...


In [365]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Unnamed: 0,1815.0,907.0,524.089687,0.0,453.5,907.0,1360.5,1814.0


In [366]:
# tweets = df['tweet']
# tweets
tweets = list(np.array(df['tweet']))

# tweets.shape
tweets

['Kata @prabowo Indonesia tidak dihargai bangsa asing!   Berita ini ðŸ‘‡ pasti hoax buatan penguasa, ya kan @rockygerung?ðŸ˜œ https://twitter.com/mediaindonesia/status/1117575436337160192?s=21Â\xa0â€¦',
 'Batuan Langka, Tasbih Jokowi Hadiah dari Habib Luthfi Seharga Mercy?  http://dlvr.it/R2pvZVÂ\xa0',
 'Di era Jokowi, ekonomi Indonesia semakin baik. #01IndonesiaMaju #JokowiLagi #JokowiMenangTotalDebat pic.twitter.com/W2ythnxsTp',
 'Bagi Sumatera Selatan, Asian Games berdampak pd ekonomi langsung diprediksi mencapai 18,5 triliun. Indonesia maju, Jokowi hebat!',
 'Negara kita ngutang buat bngun infrastruktur yang udah dipake masyarakat, terus masyarakatnya ngeluh karena negara ngutang, setiap negara itu pasti ngutang,  utang bisa dibayar kalo negara dapet penghasilan. Penghasilan negara itu ya dari pajak',
 'Yg bisikin pak jokowi, cm mikirin perputaran duit di golongan tertentu. E-Sports hanya sebagian kecil dr sekian besar penentu pertumbuhan perekonomian Indonesia, bukan hal fundament

# Text Cleaning

In [367]:
def clean_tweet(sentences):
  new_sentences = []

  # Looping setiap kalimat
  for sentence in sentences:
    # Hapus hashtag
    sentence = re.sub(r"#\w+", "", sentence)

    # hapus @
    sentence = sentence.replace('@', '')

    # Hapus karakter non-alfanumerik
    sentence = re.sub(r"[^\w\s]+", "", sentence)

    # Konversi ke representasi ASCII
    sentence = unidecode(sentence)

    # Hapus spasi ekstra
    sentence = re.sub(r"\s+", " ", sentence)

    # Trim kalimat
    sentence = sentence.strip()

    # Tambahkan kalimat baru ke dalam list
    new_sentences.append(sentence)

  return new_sentences

In [383]:
# clean_tweet(tweets)
new_sentences = []

  # Looping setiap kalimat
for sentence in tweets:
    # Hapus hashtag
    sentence = re.sub(r"#\w+", "", sentence)

    # hapus @
    sentence = sentence.replace('@', '')

    # Hapus karakter non-alfanumerik
    sentence = re.sub(r"[^\w\s]+", "", sentence)

    # Konversi ke representasi ASCII
    sentence = unidecode(sentence)

    # Hapus spasi ekstra
    sentence = re.sub(r"\s+", " ", sentence)

    # Trim kalimat
    sentence = sentence.strip()

    # Tambahkan kalimat baru ke dalam list
    new_sentences.append(sentence)

tweets = new_sentences
display(tweets[0])

'prabowo Indonesia dihargai bangsa asing Berita dY hoax buatan penguasa ya rockygerung dYoe a'

In [384]:
tweets[10]

'PilihOrangBaik PilihJelasIslamnya PilihBajuPutih Pilih pemimpin yg udh pencapaiannya 5 pencapaian ekonomi pemerintahan Jokowi Coblos 01 Yuk Kerja'

## Remove URL Link

In [369]:
# def remove_url(sentences):
#   pattern = r"(https?://.[^\s]+)"

#   return [re.sub(pattern, "", sentence) for sentence in sentences]

In [370]:
# remove_url(tweets)
pattern = r"(https?://.[^\s]+)"
tweets = [re.sub(pattern, "", sentence) for sentence in tweets]
tweets

['Kata @prabowo Indonesia tidak dihargai bangsa asing!   Berita ini ðŸ‘‡ pasti hoax buatan penguasa, ya kan @rockygerung?ðŸ˜œ \xa0â€¦',
 'Batuan Langka, Tasbih Jokowi Hadiah dari Habib Luthfi Seharga Mercy?  \xa0',
 'Di era Jokowi, ekonomi Indonesia semakin baik. #01IndonesiaMaju #JokowiLagi #JokowiMenangTotalDebat pic.twitter.com/W2ythnxsTp',
 'Bagi Sumatera Selatan, Asian Games berdampak pd ekonomi langsung diprediksi mencapai 18,5 triliun. Indonesia maju, Jokowi hebat!',
 'Negara kita ngutang buat bngun infrastruktur yang udah dipake masyarakat, terus masyarakatnya ngeluh karena negara ngutang, setiap negara itu pasti ngutang,  utang bisa dibayar kalo negara dapet penghasilan. Penghasilan negara itu ya dari pajak',
 'Yg bisikin pak jokowi, cm mikirin perputaran duit di golongan tertentu. E-Sports hanya sebagian kecil dr sekian besar penentu pertumbuhan perekonomian Indonesia, bukan hal fundamental. Betul yg dikatakan prabowo, walau dgn bhs yg sederhana, tp mrpkan aspek fundamental e

## Case Folding

In [371]:

def case_folding(sentences):
  new_sentences = []
  for sentence in sentences:
    new_sentence = sentence.lower()
    new_sentences.append(new_sentence)

  return new_sentences

In [372]:
# case_folding(df['tweet'])
# new_sentences = []
# for index in range(len(tweets)):
#     # new_sentence = sentence.lower()
#     new_sentence = case_folding(tweets[index])
#     new_sentences.append(new_sentence)

# tweets = new_sentence
# display(tweets)
tweets

['Kata @prabowo Indonesia tidak dihargai bangsa asing!   Berita ini ðŸ‘‡ pasti hoax buatan penguasa, ya kan @rockygerung?ðŸ˜œ \xa0â€¦',
 'Batuan Langka, Tasbih Jokowi Hadiah dari Habib Luthfi Seharga Mercy?  \xa0',
 'Di era Jokowi, ekonomi Indonesia semakin baik. #01IndonesiaMaju #JokowiLagi #JokowiMenangTotalDebat pic.twitter.com/W2ythnxsTp',
 'Bagi Sumatera Selatan, Asian Games berdampak pd ekonomi langsung diprediksi mencapai 18,5 triliun. Indonesia maju, Jokowi hebat!',
 'Negara kita ngutang buat bngun infrastruktur yang udah dipake masyarakat, terus masyarakatnya ngeluh karena negara ngutang, setiap negara itu pasti ngutang,  utang bisa dibayar kalo negara dapet penghasilan. Penghasilan negara itu ya dari pajak',
 'Yg bisikin pak jokowi, cm mikirin perputaran duit di golongan tertentu. E-Sports hanya sebagian kecil dr sekian besar penentu pertumbuhan perekonomian Indonesia, bukan hal fundamental. Betul yg dikatakan prabowo, walau dgn bhs yg sederhana, tp mrpkan aspek fundamental e

## Stopwords

In [373]:
def remove_stopwords(sentences):
  stop_words = set(stopwords.words('indonesian'))
  sentences_string = " ".join(sentences)
  sentences_string = " ".join([word for word in sentences_string.split() if word not in stop_words])

  return sentences_string.split()

def remove_stopwords_indonesian(text):
    # Kode untuk menghapus stopwords
    stop_words = set(stopwords.words('indonesian'))
    tokens = word_tokenize(text)
    filtered_tokens = [token for token in tokens if token.lower() not in stop_words]
    cleaned_text = ' '.join(filtered_tokens)
    return cleaned_text

In [374]:
# remove_stopwords(tweets)

# display(tweets.head())
# display(tweets.tail())

# stop_words = set(stopwords.words('indonesian'))
# new_sentences = []
# for sentences in tweets:
#     # sentences_string = " ".join(sentences)
#     sentences_string = [word for word in sentences.split() if word not in stop_words]
#     new_sentence = sentences_string

# tweets = new_sentence
# tweets
no_stopwords = []
for index in range(len(tweets)):
    cleaned_text = remove_stopwords_indonesian(tweets[index])
    no_stopwords.append(cleaned_text)

tweets = no_stopwords
tweets

['@ prabowo Indonesia dihargai bangsa asing ! Berita ðŸ ‘ ‡ hoax buatan penguasa , ya @ rockygerung ? ðŸ˜œ â€¦',
 'Batuan Langka , Tasbih Jokowi Hadiah Habib Luthfi Seharga Mercy ?',
 'era Jokowi , ekonomi Indonesia . # 01IndonesiaMaju # JokowiLagi # JokowiMenangTotalDebat pic.twitter.com/W2ythnxsTp',
 'Sumatera Selatan , Asian Games berdampak pd ekonomi langsung diprediksi mencapai 18,5 triliun . Indonesia maju , Jokowi hebat !',
 'Negara ngutang bngun infrastruktur udah dipake masyarakat , masyarakatnya ngeluh negara ngutang , negara ngutang , utang dibayar kalo negara dapet penghasilan . Penghasilan negara ya pajak',
 'Yg bisikin jokowi , cm mikirin perputaran duit golongan . E-Sports dr sekian penentu pertumbuhan perekonomian Indonesia , fundamental . yg prabowo , dgn bhs yg sederhana , tp mrpkan aspek fundamental ekonomi',
 'tenang msih ngoceh aja .. ttp jokowi harga mati ...',
 '# UASdifitnahKejiBalasDiTPS kerjasa ekonomi bilateral , multilateral doakan TPS dengarkan suara hati .

## Punctuation

In [375]:
import string

def remove_punctuation(sentences):
  punctuation = set(string.punctuation)
  for sentence in sentences:
    for char in punctuation:
      sentence = sentence.replace(char, "")

  return sentences

def remove_punctuation_regex(sentences):
  """Menghilangkan punctuation dari list kalimat."""

  # Buat regular expression untuk punctuation
  regex = re.compile(r"[^a-zA-Z0-9]")

  # Hapus punctuation dari setiap kalimat
  for sentence in sentences:
    sentence = regex.sub("", sentence)

  return sentences

In [376]:
# remove_punctuation_regex(tweets)

  # Buat regular expression untuk punctuation
regex = re.compile(r"[^a-zA-Z0-9]")

  # Hapus punctuation dari setiap kalimat
for sentence in tweets:
    sentence = regex.sub("", sentence)

# display(tweets.head())
tweets

['@ prabowo Indonesia dihargai bangsa asing ! Berita ðŸ ‘ ‡ hoax buatan penguasa , ya @ rockygerung ? ðŸ˜œ â€¦',
 'Batuan Langka , Tasbih Jokowi Hadiah Habib Luthfi Seharga Mercy ?',
 'era Jokowi , ekonomi Indonesia . # 01IndonesiaMaju # JokowiLagi # JokowiMenangTotalDebat pic.twitter.com/W2ythnxsTp',
 'Sumatera Selatan , Asian Games berdampak pd ekonomi langsung diprediksi mencapai 18,5 triliun . Indonesia maju , Jokowi hebat !',
 'Negara ngutang bngun infrastruktur udah dipake masyarakat , masyarakatnya ngeluh negara ngutang , negara ngutang , utang dibayar kalo negara dapet penghasilan . Penghasilan negara ya pajak',
 'Yg bisikin jokowi , cm mikirin perputaran duit golongan . E-Sports dr sekian penentu pertumbuhan perekonomian Indonesia , fundamental . yg prabowo , dgn bhs yg sederhana , tp mrpkan aspek fundamental ekonomi',
 'tenang msih ngoceh aja .. ttp jokowi harga mati ...',
 '# UASdifitnahKejiBalasDiTPS kerjasa ekonomi bilateral , multilateral doakan TPS dengarkan suara hati .

## Special Character

In [377]:
def remove_special_character(sentences):
  regex = re.compile(r"[^\w\s\d]")
  for sentence in sentences:
    sentence = regex.sub("", sentence)

  return sentences

In [378]:
# remove_special_character(tweets)
new_tweet = []
regex = re.compile(r"[^\w\s\d]")
for sentence in tweets:
    sentence = regex.sub("", sentence)
    new_tweet.append(sentence)

tweets = new_tweet
tweets

[' prabowo Indonesia dihargai bangsa asing  Berita ðŸ   hoax buatan penguasa  ya  rockygerung  ðŸœ â',
 'Batuan Langka  Tasbih Jokowi Hadiah Habib Luthfi Seharga Mercy ',
 'era Jokowi  ekonomi Indonesia   01IndonesiaMaju  JokowiLagi  JokowiMenangTotalDebat pictwittercomW2ythnxsTp',
 'Sumatera Selatan  Asian Games berdampak pd ekonomi langsung diprediksi mencapai 185 triliun  Indonesia maju  Jokowi hebat ',
 'Negara ngutang bngun infrastruktur udah dipake masyarakat  masyarakatnya ngeluh negara ngutang  negara ngutang  utang dibayar kalo negara dapet penghasilan  Penghasilan negara ya pajak',
 'Yg bisikin jokowi  cm mikirin perputaran duit golongan  ESports dr sekian penentu pertumbuhan perekonomian Indonesia  fundamental  yg prabowo  dgn bhs yg sederhana  tp mrpkan aspek fundamental ekonomi',
 'tenang msih ngoceh aja  ttp jokowi harga mati ',
 ' UASdifitnahKejiBalasDiTPS kerjasa ekonomi bilateral  multilateral doakan TPS dengarkan suara hati   UASdifitnahKejiBalasDiTPS',
 'Iya bener Aa

In [379]:
def remove_at_from_words(sentences):
  new_sentences = []

  # Looping setiap kalimat
  for sentence in sentences:
    words = sentence.split(" ")
    for word in words:
      # Hapus @ dari kata
      new_word = re.sub(r"@", "", word)
      words[words.index(word)] = new_word
    new_sentence = " ".join(words)
    new_sentences.append(new_sentence)

  return new_sentences

In [380]:
remove_at_from_words(tweets)

tweets

[' prabowo Indonesia dihargai bangsa asing  Berita ðŸ   hoax buatan penguasa  ya  rockygerung  ðŸœ â',
 'Batuan Langka  Tasbih Jokowi Hadiah Habib Luthfi Seharga Mercy ',
 'era Jokowi  ekonomi Indonesia   01IndonesiaMaju  JokowiLagi  JokowiMenangTotalDebat pictwittercomW2ythnxsTp',
 'Sumatera Selatan  Asian Games berdampak pd ekonomi langsung diprediksi mencapai 185 triliun  Indonesia maju  Jokowi hebat ',
 'Negara ngutang bngun infrastruktur udah dipake masyarakat  masyarakatnya ngeluh negara ngutang  negara ngutang  utang dibayar kalo negara dapet penghasilan  Penghasilan negara ya pajak',
 'Yg bisikin jokowi  cm mikirin perputaran duit golongan  ESports dr sekian penentu pertumbuhan perekonomian Indonesia  fundamental  yg prabowo  dgn bhs yg sederhana  tp mrpkan aspek fundamental ekonomi',
 'tenang msih ngoceh aja  ttp jokowi harga mati ',
 ' UASdifitnahKejiBalasDiTPS kerjasa ekonomi bilateral  multilateral doakan TPS dengarkan suara hati   UASdifitnahKejiBalasDiTPS',
 'Iya bener Aa

# Normalization

In [381]:
def stem_sentences(sentences):

  stemmer = StemmerFactory().create_stemmer()
  return [stemmer.stem(sentence) for sentence in sentences]


def lemmatize_sentences(sentences):
  lemmatizer = Sastrawi.LemmatizerFactory().create_lemmatizer()
  return [lemmatizer.lemmatize(sentence) for sentence in sentences]

### Stemmer

In [382]:
stem_sentences(tweets)

display(tweets.head())
display(tweets.tail())

KeyboardInterrupt: 

### Lemmatizer

In [ ]:
lemmatize_sentences(tweets)

display(tweets.head())
display(tweets.tail())